In [159]:
data = pd.read_csv("cleaned_repositoriesV2.csv")
original_length = len(data)
original_length

22945

In [160]:
import pandas as pd
import numpy as np
import json
import sqlalchemy as sa
from sqlalchemy import text

In [161]:
server = "server"
database = "database"
table = "table"
driver = "driver"


In [162]:
def split_data_to_json_db_csv(csv_file, json_file, db_connection_string, db_table, distribution=(0.3, 0.3, 0.4), header=True):
    # Chargement du fichier CSV
    data = pd.read_csv(csv_file)
    
    # Obtenir la longueur du fichier original
    original_length = len(data)
    
    # Mélanger les données pour la randomisation
    data = data.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Calculer le nombre de lignes pour chaque destination
    total_rows = len(data)
    json_rows = int(distribution[0] * total_rows)
    db_rows = int(distribution[1] * total_rows)
    
    # Séparer les données en JSON, base de données et CSV
    data_json = data.iloc[:json_rows, :]
    data_db = data.iloc[json_rows:json_rows + db_rows, :]
    data_csv = data.iloc[json_rows + db_rows:, :]
    
    # Sauvegarder les données au format JSON
    data_json.to_json(json_file, orient='records', lines=True)
    
    # Sauvegarder les données CSV sans en-tête
    data_csv.to_csv('cleaned_repositories.csv', index=False, header=False)
    
    # Sauvegarder les données dans la base de données
    engine = sa.create_engine(db_connection_string)
    data_db.to_sql(db_table, engine, if_exists='replace', index=False)
    
    # Afficher les statistiques après l'insertion
    with engine.connect() as conn:
        query = text(f"SELECT COUNT(*) FROM {db_table};")
        result = conn.execute(query)
        db_rows_inserted = result.scalar()
        print(f"Nombre de lignes insérées dans la base de données : {db_rows_inserted} ({(db_rows_inserted / original_length) * 100:.2f}%)")
        print(f"Nombre de lignes dans le fichier JSON : {len(data_json)} ({(len(data_json) / original_length) * 100:.2f}%)")
        print(f"Nombre de lignes dans le fichier CSV : {len(data_csv)} ({(len(data_csv) / original_length) * 100:.2f}%)")

if __name__ == "__main__":
    csv_file_path = "cleaned_repositoriesV2.csv"
    json_file_path = "cleaned_repositories.json"
    db_server = server
    db_name = database
    db_driver = driver
    db_table = table
    trusted_connection = "yes"  # Vous pouvez définir ceci sur "yes" ou "no" en fonction de votre configuration
    
    # Construire la chaîne de connexion à la base de données
    db_connection_string = f"mssql+pyodbc:///?trusted_connection={trusted_connection}&driver={db_driver}&server={db_server}&database={db_name}"
    
    # Définir la répartition (30% JSON, 30% base de données, 40% CSV)
    distribution = (0.3, 0.3, 0.4)
    
    # Appeler la fonction pour diviser les données
    split_data_to_json_db_csv(csv_file_path, json_file_path, db_connection_string, db_table, distribution, header=True)


Nombre de lignes insérées dans la base de données : 6883 (30.00%)
Nombre de lignes dans le fichier JSON : 6883 (30.00%)
Nombre de lignes dans le fichier CSV : 9179 (40.00%)
